In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# read in data
films = pd.read_csv('movies_data.csv')

In [ ]:
films.head()
films.columns
films.info()
films.describe()

# 1. What are the 3 most common ratings (1­10) for movies in the list?
# Round each rating up or down to the nearest whole number.

In [ ]:
films['rating_whole'] = films['rating'].map(lambda x: np.round(x, 0))
films['rating_whole'].value_counts()